# OOP (2024 Fall) HW1: 불변 순서쌍을 활용해 가변 순서쌍 만들기 
- 이름: 이은섭
- 학번: 20210508

명색이 과제인데 이상한 주석이 많습니다.
나중에 공부할때 보려고 달아놨습니다.



-----
## Part A: 불변 순서쌍 Pair
1. 클래스(또는 정적) 팩토리 메소드 `of`를 를 수정하여 완성하라.
    - https://velog.io/@cjh8746/%EC%A0%95%EC%A0%81-%ED%8C%A9%ED%86%A0%EB%A6%AC-%EB%A9%94%EC%84%9C%EB%93%9CStatic-Factory-Method
1. 주석 부분을 풀어 제너릭 클래스인 `Pair`가 제너릭 인터페이스 `Comparable`를 구현하도록 완성하라.
    - https://docs.oracle.com/en/java/javase/22/docs/api/java.base/java/lang/Comparable.html

In [110]:
record Pair<F extends Comparable<F>, S extends Comparable<S>>(F first, S second) implements Comparable<Pair<F,S>> {
    
    // 아래 팩토리 메소드 of가 컴파일되지 않고 있으니 컴파일되도록 수정하라 (힌트: 제너릭 메소드)
    //pair생성 하는거
    //static뒤에 <>는 어떤 자료형이든 가능하도록 하는거
    //extends 어쩌구 -> 이타입의 변수끼리 비교가능
    static <F extends Comparable<F>,S extends Comparable<S>> Pair<F,S> of (F first, S second) {
        return new Pair<>(first, second);
    }
    
    // 사전순이 되도록 정의하기. 즉 first가 더 큰 쪽이 더 크고, first가 같으면 second를 비교
    // @Override -> Comparable사용하기 위해 재정의, compareTo는 Comparable 인터페이스의 메서드 
    // 2개의 페어를 비교인지 한개의 페어속에 F랑 S를 비교하는건지 모르겠음 근데 위에 주석보면 전자같음 아니면 그냥 25점 날라가는것 이해력이 유감
    // x.compareTo(y) 에서 Pair<F,S> -> this -> x, other -> y
    public int compareTo(Pair<F,S> other) {
        if (this.first() == other.first()){
            return this.second.compareTo(other.second());
        }
        return this.first.compareTo(other.first());
    }
    
}

In [152]:
// compareTo결과가 
// 0보다 작으면 x<y
// 0이면 x==y
// 0보다 크면 x>y

// x>y인 경우, 결과예상 -> 0이상
var p1 = Pair.of(3,1);
var p2 = Pair.of(1,2);
// x==y인 경우, 결과예상 -> 0
// char도 잘된다
var p3 = Pair.of("b","d");
var p4 = Pair.of("b","d");
// x<y인 경우, 결과예상 -> 0이하
var p5 = Pair.of(1,5);
var p6 = Pair.of(3,2);

System.out.println( p1.compareTo(p2) );
System.out.println( p3.compareTo(p4) );
System.out.println( p5.compareTo(p6) );

1
0
-1


------
## Part B: 가변 순서쌍 ModifiablePair

불변 순서쌍만을 priviate 필드로 갖고 있는 가변 순서쌍 제너릭 클래스를 작성하라

가변 순서쌍은 불변 순서쌍과 마찬가지 이름의 getter 메소드가 있고 추가로 setter 메소드도 있다.

또, 가변 순서쌍도 불변 순서쌍과 마찬가지로 Comparable 제너릭 인터페이스를 구현하라.


In [162]:
// 위에서 선언했던 <F ext...>(제네릭타입)는 class에서만 선언해주면된다
class ModifiablePair< F extends Comparable<F>, S extends Comparable<S>> implements Comparable<ModifiablePair<F, S>> {
    private Pair<F,S> pair; // 이 field 말고 다른 field는 추가로 정의하지 말 것!

    //pair생성하는거 변수두개받아 pair초기화 
    ModifiablePair(F first, S second) {
        this.pair = new Pair<>(first, second);
    }
    
    // getter들은 pair에게 위임(delegate)하는 방식으로 작성하라
    // 1. getter란? 필드(속성)값을 반환하는 메서드, 객체지향에서는 캡슐화를 통해 필드 직접 접근은 안되도록하고 메서드를 이용해 읽는다고한다..
    // 여기서 필드는 private Pair<F, S> pair; 값은 first, second이다. 즉아래 FS는 first second 관련 메서드인듯
    // 2. 위임이란? 특정 작업이나 책임을 다른 객체에게 맡기는 것.
    // 이 경우에선 ModifiablePair는 pair객체에 책임을 위임하여 first second값을 가져온다
    // pair객체 first(), second()호출해서 값 반환
    public F first() {
        return pair.first();
    }
    public S second() {
        return pair.second();
    }
    
    // setter들은 적절히 정의하라 
    // setter란 초기값을 다른값으로 변환시켜주는 역할
    // 자바는 불변객체의 특성으로 인해 값을 변경하려면 새롭게 생성을 해야한다고한다.. 
    // 이부분에 대해선 추후 더 찾아볼것
    // fist는 setFirst(first)의 인자로 대체하고 second는 그대로유지
    public void setFirst(F first) {
        this.pair = new Pair<>(first, this.pair.second());
    }
    // 이건 반대로
    public void setSecond(S second) {
        this.pair = new Pair<>(this.pair.first(), second);
    }

    
    // compareTo는 pair에게 위임(delegate)하는 방식으로 작성하라
    // @Override
    public int compareTo(ModifiablePair<F, S> other) {
        return this.pair.compareTo(other.pair);
    }
    

    // 건들필요 x
    public String toString() { 
        return pair.toString(); // toString은 그냥 pair에게 위임
    }
}



In [163]:
// ModifiablePair<F,S>의 getter와 setter들이 잘 동작하는지 확인할 수 있는 적절한 테스트 코드 작성하여 실행하라.
// 그러니까 `first()`, `second()`, `setFrist(F)`, `setSecond(S)` 메소드를 모두 활용하는 예시 코드를 작성해 실행하라는 말이다
// 예시코드 
var mp1 = new ModifiablePair<>(1,2);
var mp2 = new ModifiablePair<>(2,3);
System.out.println(mp1.first()); //getter -> public F first()
System.out.println(mp1.second()); // getter -> public S second()
mp1.setFirst(3); //setter -> public void setFirst(F first)
mp1.setSecond(4); //setter ->  public void setSecond(S second)
System.out.println(mp1);
System.out.println(mp1.compareTo(mp2));

1
2
Pair[first=3, second=4]
1


In [164]:
//잘된다




/* 가변 순써상에 대해서도
  compareTo 가 잘 동작하는지
   - 왼쪽이 더 큰 경우
   - 왼쪽이 더 작은 경우
   - 같은 경우
   이런 경우를 각각 포함하도록 이렇게 최소 3개 이상의 테스트를 실행해 보라
*/

// compareTo결과가 
// 0보다 작으면 x<y
// 0이면 x==y
// 0보다 크면 x>y

// x>y인 경우, 결과예상 -> 0이상
var p1 = ModifiablePair<>(3,1);
var p2 = ModifiablePair<>(1,2);
// x==y인 경우, 결과예상 -> 0
// char도 잘된다
var p3 = ModifiablePair<>("b","d");
var p4 = ModifiablePair<>("b","d");
// x<y인 경우, 결과예상 -> 0이하
var p5 = ModifiablePair<>(1,5);
var p6 = ModifiablePair<>(3,2);

System.out.println( p1.compareTo(p2) );
System.out.println( p3.compareTo(p4) );
System.out.println( p5.compareTo(p6) );

CompilerException: 